In [1]:
import pandas as pd
import dateutil
import numpy as np

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
date_parser = lambda x: dateutil.parser.parse(x, ignoretz=True)
# Improved function to sum dataframe columns which contain nan's
def nansumwrapper(a, **kwargs):
    if np.isnan(a).all():
        return np.nan
    else:
        return np.nansum(a, **kwargs)

In [4]:
t2m = 'TTT [°C]'
rh2 = 'RH [%]'
ff  = 'ff [m/s]'
Prec = 'Precip [mm/h]'
SWD = 'SWD [W/m**2]'
LWD = 'LWD [W/m**2]'
Pres = 'PPPP [hPa]'
fcc  = 'Clouds [octa]'

In [5]:
cs_file = '../data/KGI_AWS_met_obs_V2.tab'
df = pd.read_csv(cs_file,
                 delimiter='\t', index_col=['Date/Time'],
                 parse_dates=['Date/Time'], na_values='NAN',date_parser=date_parser)

In [6]:
df_day = df.resample('1d').agg({t2m:np.mean, rh2:np.mean, ff:np.mean, 
                                SWD:np.mean, LWD:np.mean, Prec:nansumwrapper,
                                Pres:np.mean, fcc:np.mean})

In [7]:
df_day['t2m_an'] = (df_day[t2m] - df_day[t2m].mean())/df_day[t2m].std()

In [8]:
cs_file = '../data/loc_stakes_SMB_2011_2015.csv'
dfmb = pd.read_csv(cs_file,
   delimiter='\t', index_col=['Date/Time'],
    parse_dates=['Date/Time'], na_values='NAN',date_parser=date_parser)

In [9]:
labels = dfmb.drop_duplicates(subset=['Event'])

In [13]:
stake_inp = []
for i in labels['Event']:#[0:4]
    
    stake = dfmb.loc[lambda df: df['Event'] == i, :]
    str_date = stake.index.values[0]
    end_date = stake.index.values[1]
    subset = df_day.loc[str_date:end_date].copy()
    hgt_era = stake['Elevation [m]'][0]
    hgt_aws = 194.5
    subset[t2m] = subset[t2m].values + (hgt_era - hgt_aws) * -0.006
    subset['PDD'] = subset[t2m].where(subset[t2m] > 0, 0).where(subset[t2m] <= 0, 1)
    subset['stake'] = stake['Event'][0]
    subset_f = subset.groupby(['stake']).agg({t2m:np.mean, rh2:np.mean, ff:np.mean, 
                                     SWD:np.mean, LWD:np.mean, Prec:nansumwrapper,
                                     Pres:np.mean, fcc:np.mean, 't2m_an':np.mean,
                                     'PDD':nansumwrapper})
    subset_f.index = ([str_date])
    subset_f0 = []
    for i in range(len(stake)-1):
        str_date = stake.index.values[i]
        end_date = stake.index.values[i+1]
        subset = df_day.loc[str_date:end_date].copy()
        hgt_era = stake['Elevation [m]'][0]
        hgt_aws = 194.5
        subset[t2m] = subset[t2m].values + (hgt_era - hgt_aws) * -0.006
        subset['PDD'] = subset[t2m].where(subset[t2m] > 0, 0).where(subset[t2m] <= 0, 1)
        subset['stake'] = stake['Event'][0]
        subset0 = subset.groupby(['stake']).agg({t2m:np.mean, rh2:np.mean, ff:np.mean, 
                                         SWD:np.mean, LWD:np.mean, Prec:nansumwrapper,
                                         Pres:np.mean, fcc:np.mean, 't2m_an':np.mean,
                                         'PDD':nansumwrapper})
        subset0.index = ([end_date])
        subset_f0.append(subset0)
    subset_f0 = pd.concat(subset_f0)
    subset_f[[Prec, 'PDD']] = 0.0
    subset_ff = pd.concat([subset_f, subset_f0])
    subset_fff = pd.concat([stake, subset_ff], axis=1)
    stake_inp.append(subset_fff)


In [14]:
stake_inp = pd.concat(stake_inp)
stake_inp.index.name = 'Date/Time'

In [16]:
stake_inp.to_csv('../data/SMB_input_2011_2015.csv', index=True, sep='\t')